In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import geopandas
import datetime as dt
import geopy
import helper

style.use('fivethirtyeight')

plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

#### Load Data

In [2]:
# Only going to be looking at Citi Bike data from March and April of 2021
citi = helper.get_all_citi_df(start_month=3, start_year=2021)


#### Add zipcode field

In [3]:
# Load previously processed stations and their corresponding zipcodes and check for
# station id's with no corresponding zipcodes
station_zipcode = pd.read_csv(r'station_zipcode.csv',
                   index_col=0,)
zipcode_dict = dict(zip(station_zipcode.station_id, station_zipcode.zipcode))
id_to_zipcode = helper.update_zip_code_dict(citi,zipcode_dict)

# mapping statin ID to zip code
citi["start_zipcode"] = citi["Start Station ID"].map(id_to_zipcode)
citi["end_zipcode"] = citi["End Station ID"].map(id_to_zipcode)

# save station to zip code mapping for future use if there were new zip codes added.
id_to_zipcode_df = pd.DataFrame(id_to_zipcode.items(), columns=['station_id', 'zipcode'])
id_to_zipcode_df.to_csv('station_zipcode.csv')

No new ZIP codes.


#### Keep only important features.

In [4]:
# Removing unessential columns
citi = citi[['Trip Duration', 'Start Time', 
               'Start Station Name', 'Start Station Latitude',
               'Start Station Longitude', 'End Station Name',
               'End Station Latitude', 'End Station Longitude', 
               'start_zipcode', 'end_zipcode']]
# Removing trips that are over an hour and half long.
trip_duration_extreme = citi["Trip Duration"].quantile(0.99)
citi = citi[(citi["Trip Duration"] < trip_duration_extreme)].reset_index(drop=True)

# Adding hour and weekday columns.
citi['hour'] = citi['Start Time'].dt.hour
# citi['day'] = citi['Start Time'].dt.day
citi['weekday'] = citi['Start Time'].dt.weekday


#### Loading NYC maps to improve graphs.

In [5]:
# NYC zip codes.
gnyc_zip = geopandas.read_file("nyc_zip_codes/ZIP_CODE_040114.shp")  
gnyc_zip = gnyc_zip.to_crs("EPSG:4326") # axis are now in lat/long format
nyc_zip_list = list(gnyc_zip.ZIPCODE)
# NYC University/Colleges
gschool = geopandas.read_file('nyc_schools/geo_export_b7c28c3f-1653-45c2-b34c-777952dfbaef.shp')
# NYC subway stations.
gsub_stations = geopandas.read_file("nyc_subway_stations/geo_export_09ee98e9-27d8-446f-bb37-c37557e74463.shp")  
# NYC subway lines. And a color mapping for the subway lines.
gsub_lines = geopandas.read_file("nyc_subway_lines/geo_export_dcdd1f9d-0a28-4d93-8b36-619951fd4f21.shp") 
subway_color_mapping = {'G':'#6cbe45', 
                        'N':'#fccc0a', 
                        'B':'#ff6319', 
                        'L':'#a7a9ac', 
                        'A':'#0039a6', 
                        '7':'#b933ad', 
                        'J':'#996633', 
                        '1':'#ee352e', 
                        '4':'#00933c'}
# NYC Hospitals.
health = pd.read_csv(r'Health_Facility_Map.csv',)
health = health[health['Description']=='Hospital']
health = health[health['Facility Zip Code'].isin(nyc_zip_list)].reset_index(drop = True)
ghealth = geopandas.GeoDataFrame(
                health, 
                geometry=geopandas.points_from_xy(health['Facility Longitude'], 
                                                  health['Facility Latitude']))

In [6]:
# Removing all trips that go to New Jersey or random ferry islands.
citi = citi[citi['start_zipcode'].isin(nyc_zip_list)]
citi = citi[citi['end_zipcode'].isin(nyc_zip_list)]

#### visualize station trip count

In [7]:
# total number of trips in a given zip code
start_zip_count = citi['start_zipcode'].value_counts().reset_index()
end_zip_count = citi['end_zipcode'].value_counts().reset_index()
citi_zip_count = pd.merge(start_zip_count, end_zip_count,  
                              how='inner', 
                              left_on=['index'], 
                              right_on = ['index'])

citi_zip_count['total_trips'] = citi_zip_count['end_zipcode']+citi_zip_count['start_zipcode']

# only looking at top ten zip codes
graph_ =  citi_zip_count.sort_values('total_trips',ascending=False).iloc[:10]
geo_graph_ = gnyc_zip.merge(graph_, right_on='index' ,left_on='ZIPCODE')

# setting up matching colors between graphs
zips_code = list(graph_['index'])
color = ['#e15759',
         '#f28e2b',
         '#4e79a7',
         '#76b7b2',
         '#59a14f',
         '#edc948',
         '#b07aa1',
         '#ff9da7',
         '#9c755f',
         '#bab0ac'] 
color_mapping = dict(zip(zips_code,color))
# create axis
fig,axs = plt.subplots(1,2,figsize = (45, 30))
# map zip codes
zip_boundry_map = gnyc_zip.boundary.plot(color='Black', linewidth=.5,ax=axs[1])
geo_graph_.plot(color=geo_graph_["ZIPCODE"].map(color_mapping),ax=zip_boundry_map)
zip_boundry_map.axis('off')
# limit around area of interest
minx, miny, maxx, maxy = geo_graph_.total_bounds
axs[1].set_xlim(minx, maxx)
axs[1].set_ylim(miny, maxy)
# graph bar plots
g = sns.barplot(x='index',y="total_trips", 
                  data=citi_zip_count,
                  order = graph_['index'],
                  palette=color,
                  ax=axs[0]
                  )
fig.tight_layout()
# counts above bars
g.set(yticks=[])
for p in g.patches:
    rmse_ = '  {:,.0f}'.format(p.get_height())
    x = p.get_x() + p.get_width() / 2 - 0.05
    y = p.get_y() + p.get_height()+1000
    axs[0].annotate(rmse_, (x, y), size = 32, 
                    color = 'black',
               horizontalalignment='center',)
# labels
fig.suptitle('Total Trips by Zip Code', fontsize=64)
g.set(xlabel='', ylabel='',title='')
axs[0].set_xticklabels(labels = ['10009','10003','10011','10019','10002',
                                 '10014','10016','10025','10010','10018'],
                       fontsize=32)


plt.show()